In [8]:
import psycopg2
import pandas as pd

# Connection parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "rosbank",
    "user"      : "sergey",
    "password"  : "mypassword"
}
def connect(params_dic):
    
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    
    """
    Tranform a SELECT query into a pandas dataframe
    """
    
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)
column_names = ['currency_id', 'currency', 'direct_quote', 'reverse_quote', 'cross_course', 'date_']
# Execute the "SELECT *" query
history_prices = postgresql_to_dataframe(conn, "select * from traiding.history_prices", column_names)
    
history_prices.head()

Connecting to the PostgreSQL database...
Connection successful


,currency_id,currency,direct_quote,reverse_quote,cross_course,date_
0,836,AUD/USD,1.3738478,1.2907935,1.3012412,2022-10-27
1,1496,USD/JPY,32.12028,1.3419746,1.3017167,2022-10-06
2,2071,NZD/USD,37.368767,1.4996032,1.2404976,2022-10-09
3,4625,USD/JPY,40.699383,1.5114788,1.220334,2022-11-17
4,253,EUR/USD,5.13296,1.4108196,1.3962678,2022-11-23


In [ ]:
# Close the connection
conn.commit()
conn.close()

In [13]:
import os

def writer(dataframe_):
    # Создание директории в которой будут храниться результаты
    if not os.path.isdir("../results"):
        os.mkdir("../results")

    save_path = '../results'

    print(f'Saving the results in /results/')

    path = os.path.join(save_path, 'history_prices.csv')
    dataframe_.to_csv(path) 

writer(dataframe_=history_prices)

Saving the results in /results/
